In [1]:
pip install -U scikit-learn

Requirement already up-to-date: scikit-learn in /usr/local/lib/python3.6/dist-packages (0.24.2)
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install dictances

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
from __future__ import division
import json
import numpy as np
from decimal import Decimal
from numpy import expand_dims, array, exp, max
import scipy
from scipy.special import rel_entr
import csv
import statistics
from numpy import expand_dims, array, exp, max
import math
from scipy.spatial import distance
from scipy.stats import wasserstein_distance
import numpy as np
from numpy.linalg import norm
import sklearn
from tensorflow import keras
import string
from scipy.special import rel_entr
from tensorflow.keras.applications.resnet50 import ResNet50 as resnet50
from tensorflow.keras.applications.vgg16 import VGG16 as vgg16
from tensorflow.keras.applications.densenet import DenseNet169 as densenet
from tensorflow.keras.applications.inception_v3 import InceptionV3 as inceptionv3
from tensorflow.keras.applications.efficientnet import EfficientNetB1 as efficient
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2 as mobilenetv2
from tensorflow.keras.applications.mobilenet import MobileNet as mobilenet
from sklearn.metrics.pairwise import cosine_similarity
from itertools import combinations
import csv
# import the timeit module
import timeit
import tracemalloc

In [4]:
from numpy.linalg import norm
from dictances import bhattacharyya, bhattacharyya_coefficient

In [5]:
# import the timeit module
import timeit
import math
import tracemalloc

In [6]:
def hellinger_fast(p, q):
    """Hellinger distance between two discrete distributions.
       In pure Python.
       Fastest version.
    """
    return sum([ (math.sqrt(p_i) - math.sqrt(q_i))**2 for p_i, q_i in zip(p,q) ])

In [7]:
# to suppress division errors
np.seterr(divide='ignore', invalid='ignore')
new_final_source_array = []

In [40]:
# Opening final JSON files
contrast_source_data = open('../paper3/files/glcm/source_domain_data_new_energy_glcm.json')

#for target data
contrast_target_data = open('files/Caltech256/DenseNet169/glcm/caltech_domain_data_new_energy_glcm.json')

In [41]:
# returns JSON object
contrast_source_data_d = json.load(contrast_source_data)

contrast_target_data_d = json.load(contrast_target_data)

In [42]:
# kdl dictionary
kdl_dictionary = {}

In [43]:
def KL(p, q):
    a = []
    b = []
    for k in p:
        a.append(Decimal(k))

    for l in q:
        b.append(Decimal(l))

    a = np.asarray(a, dtype=np.float64)
    b = np.asarray(b, dtype=np.float64)

    return np.sum(np.where(((a != 0) & (b != 0)), a * np.log(a / b), 0))

In [44]:
def softMax(temperature, features_list):
    feat_list = [x / temperature for x in features_list]
    feature_list = exp(feat_list - max(feat_list))
    return feature_list / feature_list.sum(axis=0)

In [45]:
# get the data for the contrast target image
contrast_target_data_array = []
for p in range(0, len(contrast_target_data_d['target_images'])):
    for i, j in contrast_target_data_d['target_images'][p].items():
        contrast_target_data_array.append((i,np.array(j)))
contrast_target_data.close()

In [46]:
print(len(contrast_target_data_array))

7710


In [47]:
###contrast comparison code
contrast_All_Images = []
contrast_average_dkl = []

#check the timer_start
start_hellinger = timeit.default_timer()
#check memory consumption
# starting the monitoring
tracemalloc.start()

for x in contrast_target_data_array[:1]:
#for x in target_data_array:
    t_images_btween_0_05 = []
    t_images_btween_05_1 = []
    t_images_btween_1_2 = []
    t_images_grt_2_5 = []
    t_images_grt_5_10 = []
    t_images_grt_10 = []
    
    image_name_class =  x[0].split('_/') #name[0]; class[1]
    compare_min_max = []
    
    
            
    # get the data for the source images
    for m in range(0, len(contrast_source_data_d['source_images'])):
        #print("source",source_data_d['source_images'][m][0])
        for k, v in contrast_source_data_d['source_images'][m].items():
            # compare the value of v with that of the target image
            # if 'caterpillar' in k.lower():
            # remove the zeros from the list
            new_v = np.array(v).flatten()
            #get the top items in x
            items = np.sort(x[1])
            item_size = items[-len(new_v):]
            new_x = item_size.flatten()
            
            new_d = softMax(0.5, new_x)     
            new_y = softMax(0.5, new_v)
            
            kdl = rel_entr( new_x, new_v, out=None)
            new_bhattacharyya = bhattacharyya(dict(enumerate(new_x.flatten(), 1)),dict(enumerate(new_v.flatten(), 1)) )
            new_wasserstein = wasserstein_distance(new_x,new_v)
            new_jensen = distance.jensenshannon(new_x,new_v)
            new_hellinger = hellinger_fast(new_x,new_v)
            
            compare_min_max.append((image_name_class[0],image_name_class[1],k, max(new_bhattacharyya)))
            
contrast_All_Images.append(compare_min_max)
    
contrast_source_data.close()

stop = timeit.default_timer()
execution_time = stop - start_hellinger
print("KDL -selected points execution time:-", execution_time)

# displaying the memory
print("used memory",tracemalloc.get_traced_memory()[1]-tracemalloc.get_traced_memory()[0])
 
# stopping the library
tracemalloc.stop()

model_name = densenet
input_t = keras.Input(shape=(224, 224, 3))
model = model_name(include_top=False,
                   weights="imagenet",
                   input_tensor=input_t)


# get the layer index
def getLayerIndex(model_i, layer_name):
    for pos, layer_g in enumerate(model_i.layers):
        if layer_g.name == layer_name:
            return pos


# get the convolved layers into an array for looping
convolved_layers = []
convolved_layer_names = []
# create a list to place them
layer_cosine_list = []
layer_final_cos_list = []

for layer in model.layers[0:-2]:
    if 'conv' not in layer.name:
        continue

    index = getLayerIndex(model, layer.name)
    #print(layer.name)
    # append the convolved layer
    convolved_layers.append(index)
    convolved_layer_names.append(layer)

jdk = {}
for lyr in convolved_layer_names:
    ary = np.array(lyr.get_weights(), dtype=object)
    layer_index = getLayerIndex(model, lyr.name)
    if len(ary) != 0:
        # filters, biases = layer.get_weights()
        # print(filters.shape)
        ary = np.array(lyr.get_weights(), dtype=object)

        zipper_dict = {}
        dict_array = []

        # check for the arrays
        for x in ary:
            # print(len(x))
            # find if the array is 1 dim
            if x.ndim > 1:
                for y in x:
                    for z in y:
                        u = 0
                        for za in z:
                            # get all the positive elements in this and push them into an array
                            positive_elements = []
                            for pos_item in za:
                                if pos_item > 0:
                                    positive_elements.append(pos_item)
                            u = u + 1
                            zipper_dict.update({u: positive_elements})

            elif x.ndim == 1:
                # get the items/arrays with more than the 1 dimension
                print("")
        # print("lyr:",lyr.name)
        # get two items per tuple
        for i in range(1, len(zipper_dict), 2):
            if len(zipper_dict[i]) > 0:
                if len(zipper_dict[i]) == len(zipper_dict[i + 1]):

                    new_i = np.array(zipper_dict[i]).reshape(1, -1)
                    new_i_plus_1 = np.array(zipper_dict[i + 1]).reshape(1, -1)
                    cos_item = np.ravel(cosine_similarity(new_i, new_i_plus_1))
                    layer_cosine_list.append(cos_item)

                elif len(zipper_dict[i]) > len(zipper_dict[i + 1]):

                    diff_len = len(np.array(zipper_dict[i])) - len(np.array(zipper_dict[i + 1]))
                    old_i = np.pad(zipper_dict[i + 1], (0, diff_len), 'constant')

                    new_i = old_i.reshape(1, -1)
                    new_i_plus_1 = np.array(zipper_dict[i]).reshape(1, -1)

                    cos_item = np.ravel(cosine_similarity(new_i, new_i_plus_1))
                    layer_cosine_list.append(cos_item)

                elif len(zipper_dict[i]) < len(zipper_dict[i + 1]):
                    # print("######less than i plus 1#########")

                    diff_len = len(zipper_dict[i + 1]) - len(zipper_dict[i])
                    old_i = np.pad(zipper_dict[i], (0, diff_len), 'constant')
                    new_i = old_i.reshape(1, -1)
                    new_i_plus_1 = np.array(zipper_dict[i + 1]).reshape(1, -1)

                    cos_item = np.ravel(cosine_similarity(new_i, new_i_plus_1))
                    layer_cosine_list.append(cos_item)


        # calculate softmax function
        def softmax(s_et):
            e_set = np.exp(s_et - np.max(s_et))
            return e_set / e_set.sum(axis=0)


        for a in layer_cosine_list:
            layer_final_cos_list.append(a)
        # print(layer_final_cos_list)
        layer_softmax_values = softmax(np.ravel(np.array(layer_cosine_list, dtype=np.float32).reshape(1, -1)))
        jdk.update({layer_index: layer_softmax_values})

# jdk gives a dictionary made up of probability distributions
# print(jdk)
# print("the other one is")
b = [x for y, x in enumerate(jdk) if y != i]
all_items = list(combinations(b, 2))

# put the layers in a dictionary and rank them --JENSEN
other = []
#check timer here
# start the timer
start_jensen = timeit.default_timer()
#check memory consumption
# starting the monitoring
tracemalloc.start()

       
for a in all_items:
    if len(jdk[a[0]]) > len(jdk[a[1]]):
        
        dif_1 = len(np.array(jdk[a[0]])) - len(np.array(jdk[a[1]]))
        new_jdk = np.pad(np.array(jdk[a[1]]), (0, dif_1), 'constant').reshape(1, -1)
        new_i_jdk = np.array(jdk[a[0]]).reshape(1, -1)
        #print("########################other###############") 
        new_jensen = distance.jensenshannon(new_jdk[0],new_i_jdk[0])
        ##print(new_jensen)
        #print("##########################end other#########")
        
        #hellinger_fast(new_i_jdk, new_jdk)
        ##kl_pq = rel_entr(np.ravel(np.ravel(new_i_jdk), new_jdk))
       # print("layer:", a[0], " and layer: ", a[1], ':KL(P ||2 Q): %.3f nats1' % sum(kl_pq))
        ##########m_no = [a[0], a[1], hellinger_fast]
        ##########other.append(m_no)

    elif len(jdk[a[0]]) < len(jdk[a[1]]):
        #check timer here
        dif_1 = len(np.array(jdk[a[1]])) - len(np.array(jdk[a[0]]))
        new_jdk = np.pad(np.array(jdk[a[0]]), (0, dif_1), 'constant').reshape(1, -1)
        new_i_jdk = np.array(jdk[a[1]]).reshape(1, -1)
        #print("########################other###############")        
        new_jensen = distance.jensenshannon(new_jdk[0],new_i_jdk[0])
        
        ##print(new_jensen)
        #print("##########################end other#########")       
        

        #kl_pq = rel_entr(np.ravel(new_jdk), np.ravel(new_i_jdk))
        #print("layer:", a[0], " and layer: ", a[1], ':KL(P ||2 Q): %.3f nats' % sum(kl_pq))
        #m_no = [a[0], a[1], sum(kl_pq)]
        ###########hellinger_fast(new_i_jdk, new_jdk)
        ##kl_pq = rel_entr(np.ravel(np.ravel(new_i_jdk), new_jdk))
       # print("layer:", a[0], " and layer: ", a[1], ':KL(P ||2 Q): %.3f nats1' % sum(kl_pq))
        ###########m_no = [a[0], a[1], hellinger_fast]
        ##############other.append(m_no)
stop_jensen = timeit.default_timer()
execution_time = stop_jensen - start_jensen
print("Jensen execution time:-", execution_time)

# displaying the memory
print("used memory",tracemalloc.get_traced_memory()[1]-tracemalloc.get_traced_memory()[0])
 
# stopping the library
tracemalloc.stop()
# with open('mobilenet_kullback_positive.csv', 'w', encoding='UTF8', newline='') as f:
#     writer = csv.writer(f)  # write the header
#     writer.writerow(["Layer1", "Layer2", "DKL"])
#     for l_item in other:
#         writer.writerow(l_item)
    # All the program statements


KDL -selected points execution time:- 4.678331363014877
used memory 12278




















































































































































































































































































































































































































































































































































































































































































Jensen execution time:- 74.26523017702857
used memory 775056
